<a href="https://colab.research.google.com/github/projectsforstudents2022/Classify_Nationalities/blob/main/Classify_Nationalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import os
import re

In [18]:
# Mount Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
male_data = pd.read_csv('/content/drive/MyDrive/Projects/Classify_Nationalities/Indian-Male-Names.csv')
female_data = pd.read_csv('/content/drive/MyDrive/Projects/Classify_Nationalities/Indian-Female-Names.csv')

In [20]:
female_data.head()

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian


In [21]:
repl_list = ['s/o','d/o','w/o','/','&',',','-']

def clean_data(name):
	name = str(name).lower()
	name = (''.join(i for i in name if ord(i)<128)).strip()
	for repl in repl_list:
		name = name.replace(repl," ")
	if '@' in name:
		pos = name.find('@')
		name = name[:pos].strip()
	name = name.split(" ")
	name = " ".join([each.strip() for each in name])
	return name

def remove_records(merged_data):
	merged_data['delete'] = 0
	merged_data.loc[merged_data['name'].str.find('with') != -1,'delete'] = 1	
	merged_data.loc[merged_data['count_words']>=5,'delete']=1
	merged_data.loc[merged_data['count_words']==0,'delete']=1
	merged_data.loc[merged_data['name'].str.contains(r'\d') == True,'delete']=1
	cleaned_data = merged_data[merged_data.delete==0]
	return cleaned_data

merged_data = pd.concat((male_data,female_data),axis=0)

merged_data['name'] = merged_data['name'].apply(clean_data)
merged_data['count_words'] = merged_data['name'].str.split().apply(len)

cleaned_data = remove_records(merged_data)

indian_cleaned_data = cleaned_data[['name','count_words']].drop_duplicates(subset='name',keep='first')
indian_cleaned_data['label'] = 'indian'

len(indian_cleaned_data)

13754

In [22]:
cleaned_data.head()

,name,gender,race,count_words,delete
0,barjraj,m,indian,1,0
1,ramdin verma,m,indian,2,0
2,sharat chandran,m,indian,2,0
3,birender mandal,m,indian,2,0
4,amit,m,indian,1,0


In [23]:
merged_data.race.value_counts()

indian    30227
Name: race, dtype: int64

In [24]:
indian_cleaned_data.head()

,name,count_words,label
0,barjraj,1,indian
1,ramdin verma,2,indian
2,sharat chandran,2,indian
3,birender mandal,2,indian
4,amit,1,indian


In [25]:
!pip install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from faker import Faker
fake = Faker('en_US')
fake.name()

'Gina Thomas'

In [27]:
from faker import Faker
import random
req = 15000
non_indian_names = []

langs = ['ar_EG','bs_BA','de_DE','dk_DK','en_AU','en_CA','en_GB',
'en_IN','en_NZ','en_US','it_IT','no_NO','ro_RO']

for i in range(0,req):
	lng_indx = random.randint(0,len(langs)-1)
	fake = Faker(langs[lng_indx])
	non_indian_names.append(fake.name().lower())

non_indian_names_orig = list(set(non_indian_names))


In [28]:
len(non_indian_names_orig)

14564

In [29]:
non_indian_data = pd.DataFrame({'name':non_indian_names_orig})
non_indian_data['count_words'] = non_indian_data['name'].str.split().apply(len)
non_indian_data.head()

,name,count_words
0,samantha terry,2
1,luciano foscari,2
2,lauretta cerutti,2
3,frau oxana scheibe,3
4,debra kelly,2


In [30]:
indian_cleaned_data['count_words'].value_counts()

2    7954
1    4322
3    1344
4     134
Name: count_words, dtype: int64

In [31]:
non_indian_data['count_words'].value_counts()

2    13221
3     1149
4      186
5        8
Name: count_words, dtype: int64

In [32]:
two_word_names = non_indian_data[non_indian_data['count_words']==2]['name']
one_word_req = 5000
names_one_two_words = [each.split()[0] for each in two_word_names[:one_word_req]] + list(two_word_names[one_word_req:])
count_words = [1] * one_word_req + [2] * len(two_word_names[one_word_req:])
not_two_words_pd  = non_indian_data[non_indian_data['count_words']!=2]
one_two_words_pd = pd.DataFrame({'name':names_one_two_words,'count_words':count_words})
non_indian_data = pd.concat((not_two_words_pd,one_two_words_pd),axis=0)
non_indian_data['count_words'].value_counts()
non_indian_data['label'] = 'non_indian'
non_indian_data = non_indian_data[non_indian_data['count_words']<5]
non_indian_data['count_words'].value_counts()

2    8221
1    5000
3    1149
4     186
Name: count_words, dtype: int64

In [33]:
full_data = pd.concat((non_indian_data[['name','label']],indian_cleaned_data[['name','label']]),axis=0)
name_data = full_data.sample(frac=1)
name_data.head()

,name,label
1979,antim,indian
1058,james,non_indian
694,naseeba,indian
6640,suresh singh,indian
10170,albert harris,non_indian


In [34]:
name_data['label'].value_counts()

non_indian    14556
indian        13754
Name: label, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
X = name_data['name'].astype(str)
Y = name_data['label']
train_names,test_names,train_labels,test_labels = train_test_split(X,Y,test_size=0.20,random_state =42,stratify=Y)

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report

vectorizer = CountVectorizer()
X_ = vectorizer.fit_transform(train_names.values.astype('U'))
len(vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


11553

In [37]:
model = MultinomialNB()
model.fit(X_,train_labels)

X_test = vectorizer.transform(test_names.values.astype('U'))

test_predicted = model.predict(X_test)

print(classification_report(test_labels,test_predicted))

              precision    recall  f1-score   support

      indian       0.98      0.77      0.86      2751
  non_indian       0.82      0.99      0.90      2911

    accuracy                           0.88      5662
   macro avg       0.90      0.88      0.88      5662
weighted avg       0.90      0.88      0.88      5662



In [38]:
check_new_names = ['lalitha','tyson','shailaja','shyamala','vishwanathan','ramanujam','conan','kryslovsky',
'ratnani','diego','kakoli','shreyas','brayden','shanon']

X_new = vectorizer.transform(check_new_names)
predictions_nb_cv = model.predict(X_new)
test = pd.DataFrame({'names':check_new_names,'predictions_nb_cv':predictions_nb_cv}) 
test

,names,predictions_nb_cv
0,lalitha,non_indian
1,tyson,non_indian
2,shailaja,non_indian
3,shyamala,non_indian
4,vishwanathan,non_indian
5,ramanujam,non_indian
6,conan,non_indian
7,kryslovsky,non_indian
8,ratnani,non_indian
9,diego,non_indian
